In [1]:
experiment_name= "cars_resnet34_randomhard_margin_p5"

In [2]:
import glob
import cv2

import matplotlib.pyplot as plt
%matplotlib inline
from pandas.core.common import flatten
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import datasets
import networks

##### Cars dataset downloaded from: https://www.kaggle.com/datasets/jutrera/stanford-car-dataset-by-classes-folder/code?resource=download
This version of the dataset already has all the images in the proper training and validation folders.

In [3]:
#######################################################
#                  Create Dataset
#######################################################
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize(256),
                                transforms.CenterCrop(256),
                                transforms.ToTensor()
                               ,
                              transforms.Normalize(mean, std)
                               ])

train_dataset = datasets.CarsDataset(transform, train=True)
test_dataset = datasets.CarsDataset(transform, train=False)

In [4]:
test_dataset.labels

array([  0,   0,   0, ..., 195, 195, 195])

In [5]:
# check if CUDA is available
cuda = torch.cuda.is_available()

if not cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [6]:
# Load model and optimizer from disk:
embedding_net= torchvision.models.resnet34(pretrained=True)
embedding_net.fc= nn.Sequential(nn.Linear(512, 128),
                                nn.BatchNorm1d(128),
                                nn.ReLU(),
                                nn.Linear(128, 128), # embedding size
                                )

model= networks.TripletNet(embedding_net)

import torch.optim as optim
optimizer = optim.Adam(model.embedding_net.fc.parameters(), lr=1e-3)

PATH= experiment_name+'.pt'
check_point= torch.load(PATH)

model.embedding_net.load_state_dict(check_point['model_state_dict'])

/home/ayla/anaconda3/envs/fashion/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ayla/anaconda3/envs/fashion/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [7]:
device = torch.device('cuda')
model.to(device)

TripletNet(
  (embedding_net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

https://github.com/littleredxh/EasyPositiveHardNegative

In [8]:
import torch
from torch.utils.data.sampler import SequentialSampler
import torch.nn.functional as F

def feature(dataset, model):
    Fvecs = []
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=400, sampler=SequentialSampler(dataset))
    torch.set_grad_enabled(False)
    model.eval()
    for data, target in dataLoader:
        #inputs_bt, labels_bt = data # <FloatTensor> <LongTensor>
        #fvec = model(inputs_bt.cuda())
        
        # data.shape: [400, 3, 224, 224]; target.shape: [400] 
        
        if cuda:
            data = data.to(device)
        
        fvec = model.get_embedding(data) # fvec.shape: [400, 128]

        fvec = F.normalize(fvec, p = 2, dim = 1).cpu() 
        # F.normalize:  performs L2 normalization (p=2) over each row vector (dim=1) 
        # by first calculating the norm of each vector to find max norm; then divides each vector to this max norm.
        # Also known as Euclidean norm
        
        Fvecs.append(fvec)
        
    return torch.cat(Fvecs,0) # Concatenates the given sequence of seq tensors in the given dimension.

In [9]:
Fvectors = feature(test_dataset, model)

In [10]:
len(Fvectors)

8041

In [11]:
Fvectors.shape

torch.Size([8041, 128])

In [12]:
def recall(Fvec, imgLab,rank=None):
    # for each row in 

    # imgLab numpy.ndarray of shape: (8041,)
   
    N = len(imgLab) #8041 labels

    imgLab = torch.LongTensor([imgLab[i] for i in range(len(imgLab))])
    # imgLab.shape: [8041]
    # Fvec.shape: [8041, 128]
    
    D = Fvec.mm(torch.t(Fvec)) # mm: matrix multiplication. (n×m) mm (m×p) results in  (n×p) tensor.
    # [8041, 128] mm [128, 8041] --> [8041, 8041] this is D matrix
    # There are 1's along the diagonal!
    
    D[torch.eye(len(imgLab)).bool()] = -1 
    # torch.eye: Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    # D[torch.eye(len(imgLab)).bool()]: diagonal elements of D will take a value of -1 ; the rest will remain the same
    
    if rank==None: # only rank 1 is computed
        _,idx = D.max(1) # returns both values and indices; dim=1 means returns for each row 
        imgPre = imgLab[idx]
        A = (imgPre==imgLab).float()
        return (torch.sum(A)/N).item()
    else:
        _,idx = D.topk(rank[-1])
        acc_list = []
        for r in rank:
            A = 0
            for i in range(r):
                imgPre = imgLab[idx[:,i]]
                A += (imgPre==imgLab).float()
            acc_list.append((torch.sum((A>0).float())/N).item())
        return torch.Tensor(acc_list)

In [13]:
recall(Fvectors, test_dataset.labels, rank= [1, 2, 4, 8, 10, 100]) 

tensor([0.1970, 0.2977, 0.4237, 0.5605, 0.6039, 0.9202])

In [14]:
recall(Fvectors, test_dataset.labels) # recall@1 equals to accuracy

0.19699043035507202

In [57]:
# Let's see how recall for K=1 is calculated:

In [17]:
Fvectors.shape

torch.Size([8041, 128])

In [18]:
D = Fvectors.mm(torch.t(Fvectors))

In [19]:
D.shape

torch.Size([8041, 8041])

In [20]:
D

tensor([[1.0000, 0.5458, 0.8361,  ..., 0.2928, 0.2820, 0.3293],
        [0.5458, 1.0000, 0.7865,  ..., 0.8662, 0.6513, 0.7955],
        [0.8361, 0.7865, 1.0000,  ..., 0.6743, 0.3592, 0.4245],
        ...,
        [0.2928, 0.8662, 0.6743,  ..., 1.0000, 0.7028, 0.7852],
        [0.2820, 0.6513, 0.3592,  ..., 0.7028, 1.0000, 0.9320],
        [0.3293, 0.7955, 0.4245,  ..., 0.7852, 0.9320, 1.0000]])

In [21]:
Diag= torch.eye(len(test_dataset.labels))

In [22]:
D[Diag.bool()]=-1

In [23]:
D

tensor([[-1.0000,  0.5458,  0.8361,  ...,  0.2928,  0.2820,  0.3293],
        [ 0.5458, -1.0000,  0.7865,  ...,  0.8662,  0.6513,  0.7955],
        [ 0.8361,  0.7865, -1.0000,  ...,  0.6743,  0.3592,  0.4245],
        ...,
        [ 0.2928,  0.8662,  0.6743,  ..., -1.0000,  0.7028,  0.7852],
        [ 0.2820,  0.6513,  0.3592,  ...,  0.7028, -1.0000,  0.9320],
        [ 0.3293,  0.7955,  0.4245,  ...,  0.7852,  0.9320, -1.0000]])

In [24]:
values, indices =D.max(1)

In [25]:
values

tensor([0.9837, 0.9986, 0.9884,  ..., 0.9832, 0.9963, 0.9698])

In [26]:
indices

tensor([5470, 5300, 7416,  ..., 7681, 8003, 5165])

In [42]:
labels_predicted_as_same= test_dataset.labels[indices]

In [44]:
is_correct= labels_predicted_as_same == test_dataset.labels

In [45]:
is_correct

array([False, False, False, ..., False,  True, False])

In [51]:
ratio_corrects= sum(is_correct) / len(test_dataset.labels)

In [52]:
ratio_corrects

0.32856609874393733

In [27]:
rank= [1,5,10]

_,idx = D.topk(rank[-1])


In [28]:
idx # the most similar 10 indices are returned

tensor([[5470, 4341, 1982,  ..., 5250, 6854, 1984],
        [5300, 6974, 8025,  ..., 2400, 6985, 6979],
        [7416, 3030, 1712,  ...,  560,  808, 5209],
        ...,
        [7681, 7662, 8006,  ..., 5151, 6836, 5178],
        [8003, 2069,  732,  ..., 1540, 6957, 5279],
        [5165, 8030, 7999,  ...,  736, 3677, 5149]])

In [29]:
idx.shape

torch.Size([8041, 10])

In [51]:
test_dataset.labels.shape

(8041,)

In [53]:
test_dataset.labels[idx[:,0]]

array([133, 129, 180, ..., 187, 195, 126])

In [54]:
test_dataset.labels[idx[:,1]]

array([106, 170,  74, ..., 187,  50, 195])

In [55]:
test_dataset.labels[idx[:,2]]

array([ 48, 195,  41, ..., 195,  17, 195])

In [56]:
test_dataset.labels[idx[:,3]]

array([ 44, 172,  26, ..., 195, 148, 195])

In [57]:
test_dataset.labels[idx[:,4]]

array([ 48, 170, 194, ..., 115, 148, 170])

In [59]:
test_dataset.labels[idx[:,4]]==test_dataset.labels

array([False, False, False, ..., False, False, False])

In [58]:
N= len(test_dataset.labels)
acc_list = []
for r in rank:
    print("rank: "+ str(r))
    A = 0
    for i in range(r):
        print("\t i: "+ str(i))
        imgPre = test_dataset.labels[idx[:,i]]
        print("\t"+ str(imgPre))
        A += (imgPre==test_dataset.labels)
        print("\t A Array: " + str(A))
    ratio_correct= sum((A>0))/N
    print("Correct ratio:"+ str(ratio_correct))
    acc_list.append(ratio_correct)
print(acc_list)

rank: 1
	 i: 0
	[133 129 180 ... 187 195 126]
	 A Array: [0 0 0 ... 0 1 0]
Correct ratio:0.3284417361024748
rank: 5
	 i: 0
	[133 129 180 ... 187 195 126]
	 A Array: [0 0 0 ... 0 1 0]
	 i: 1
	[106 170  74 ... 187  50 195]
	 A Array: [0 0 0 ... 0 1 1]
	 i: 2
	[ 48 195  41 ... 195  17 195]
	 A Array: [0 0 0 ... 1 1 2]
	 i: 3
	[ 44 172  26 ... 195 148 195]
	 A Array: [0 0 0 ... 2 1 3]
	 i: 4
	[ 48 170 194 ... 115 148 170]
	 A Array: [0 0 0 ... 2 1 3]
Correct ratio:0.6494217137171994
rank: 10
	 i: 0
	[133 129 180 ... 187 195 126]
	 A Array: [0 0 0 ... 0 1 0]
	 i: 1
	[106 170  74 ... 187  50 195]
	 A Array: [0 0 0 ... 0 1 1]
	 i: 2
	[ 48 195  41 ... 195  17 195]
	 A Array: [0 0 0 ... 1 1 2]
	 i: 3
	[ 44 172  26 ... 195 148 195]
	 A Array: [0 0 0 ... 2 1 3]
	 i: 4
	[ 48 170 194 ... 115 148 170]
	 A Array: [0 0 0 ... 2 1 3]
	 i: 5
	[ 72 170  24 ... 172 115 172]
	 A Array: [0 0 0 ... 2 1 3]
	 i: 6
	[ 51 126 127 ... 110  50 195]
	 A Array: [0 0 0 ... 2 1 4]
	 i: 7
	[128  58  13 ... 126  37  17]
